## 課題1

- 課題2が終わってから`internal/apiserver/echo.go`にエンドポイントを追加した
```go
	e.GET("/", func(c echo.Context) error {
		return c.String(200, "AWS")
	})
```

## 課題2

### 1. `auth` モジュールの実装

- **目的**: 認証サーバーとバックエンドサーバーを分離し、バックエンドではトークンの検証のみを行う設計にする。
- **実施内容**:
    - バックエンドで必要な認証処理を行うミドルウェアを作成。
        - 過去に JWT 認証で使用したエンティティ (`Token`, `Claims`) とインターフェース (`TokenService`) を `internal/auth/*` に再利用。
        - ミドルウェア `internal/auth/echomiddleware.go` をDigest認証に合わせて微修正。

### 2. `iam` モジュールの実装

- **目的**: Digest 認証では完全なステートレス化が難しく、認証とログインの分離が困難であるため、認証関連の処理を `iam` (Identity and Access Management) モジュールとして分離。
- **実施内容**:
    - `internal/iam/digest.go` で初期の nonce 発行 (`Init`) とトークンのパース (`Parse`) を行う `Digest` 構造体を開発。
    - 以下のインターフェースを定義:
        - アカウント情報を取得する `AccountRepo`。
        - リプレイ攻撃防止のための nonce カウント (`Nc`) を管理・検証する `DigestNcRepo`。
        - nonce の生成と改ざん防止のための検証を行う `NonceService`。
        - クライアントからの Digest レスポンスを検証する `DigestTokenValidator`。
    - 具体的なサービスを実装:
        - MD5 を用いてトークンを検証する `MD5DigestValidator` (`internal/iam/validator.go`)。
        - nonce カウント (`Nc`) をメモリ上で管理する `InMemoryDigestNcRepo` (`internal/iam/inmemory.go`)。
        - HMAC を用いて nonce を生成・検証する `HMACNonceService` (`internal/iam/nonceservice.go`)。
        - アカウント情報を取得する `InMemoryAccountRepo` (`internal/iam/inmemory.go`)。
    - Digest 認証のチャレンジとレスポンスの流れを処理するミドルウェア `EchoDigestMiddleware` を実装 (`internal/iam/echodigestmiddleware.go`)。
    - Nc等の解放していないリソースがあるから修正いる

### 3. `apiserver` モジュールの実装

- **目的**: API サーバーを設定し、依存性の注入とルートの登録を行う。
- **実施内容**:
    - `internal/apiserver/echo.go` にて:
        - Echo サーバーの初期化。
        - CORS 設定の構成。
        - `iam` の各コンポーネント（アカウントリポジトリ、nonce サービス、バリデータなど）の依存性注入を設定。
        - ミドルウェアチェーンに `iam.EchoDigestMiddleware` と `auth.EchoMiddleware` を追加。
        - ルート（例：`/secret` エンドポイント）の登録。
        - サーバーの起動。

### 4. 参考文献

- [Digest認証の仕様](https://datatracker.ietf.org/doc/html/rfc7616)
    - `The nc value is the hexadecimal count of the number of requests (including the current request) that the client has sent with the noncevalue in this request. `
- [Digest認証日本語訳](https://tex2e.github.io/rfc-translater/html/rfc7616.html)
- [Digest認証日本語解説](https://kunishi.gitbook.io/web-application-textbook/storage)
- [RubyのDigest Client](https://www.rubydoc.info/gems/net-http-digest_auth/1.1.1/Net/HTTP/DigestAuth)

## 課題3

### stockモジュールの実装

- **目的**: 在庫管理システムのドメインに基づき、EventSourcing と CQRS を採用した stock モジュールを実装。
  
- **ドメイン設計を行った**:
    - `stock`モジュール内で「stock」がすべての主語となる命名にした。
        - 例: Event は `StockEvent`、Aggregate は `StockAggregate`、Add 関数は `AddStock` という形で、主語が省略されている。
        - Repository Pattern に倣い、具体的なプレフィックスがつく構造。例えば `UserRepository` と同じように考えているけど、主語が省略されている。
    - `producer` や `consumer` の設計も Repository Pattern と類似。ただし、`command service` 限定の処理に限定。
    - `query service` は、projection された read model に対して repository pattern を用いてデータを取得する簡易な設計。

- **projection の取り扱いを考えた**:
    - Projection を別サービスにすることも検討したが、command service が aggregate した結果を用いれば十分と判断。
    - そのため、`command service` が projection の処理を兼任し、aggregate の結果を produce。
    - Kafka Connect を用いて、aggregate のトピックを consume し、read model に反映する予定だが、いったんmockを作って全部動作確認した。
    
- **Mockサービスを実装**:
    - EventProducerを実装
    - EventConsumerを実装
    - Repositoryを実装
    - すべてInMemoryEventStoreとして実装した

- **price と sales の扱いを考え直した**:
    - もともと文字列だったが、480.0という数字にする必要があることに気づいた
    - 金額のような精度が求められる計算では浮動小数点による誤差を考える必要があることを思い出した
    - 高精度な計算が必要と判断し、decimal ライブラリを使用して金額計算を実装しなおした
    
- **command serviceで必要なインフラを準備**:
    - dockerを使用してredpandaとredpanda consoleを立てた
    - zaiko.stockのzaoko.stock_projectionの二つのtopicとschemaを用意した
    - strategyはtopic&record, avroを使用
        - zaiko.stockは在庫管理における様々なイベントを持ち、すべて型が異なる
        - zaoko.stock_projectionは拡張性を考えて設計
    
- **具体的なサービスを実装**:
    - KafkaProducerを実装
        - avroのschemaのkeyはsubにすべきなので、すべてのentityが正しくsubを持つよう修正、echorouteではMockの認証claimをセットするようにした
    - KafkaConsumerを実装
    - Query Serviceのデータベースとしてmysqlを採用
        - Elasticsearch, Meilisearch等を調べてどれにするか迷った
        - 最終的に全文検索エンジンは必要ないと考えた
    - Grafanaによるメトリクスサービスを構築

### 確認ポイント

- **HTTP メソッドと API の理解**:
    - HTTP メソッド（GET、POST、DELETE など）の使い方と意味を理解。
    - curl コマンドの `-d` オプションで POST リクエストを送信できることを学習。
  
- **API の実行と確認**:
    - 実装した5つのAPIを、curl コマンドを用いて同じ結果が得られることを確認。
    - 異常系（不正な値のリクエストやエラー処理）にも対応するよう実装。
    - 価格の計算精度や変数ごとの 0 の扱いに注意し、関連しないメソッドに対してもエラーを出すように設定。

### 参考文献

- [プログラムの計算精度](https://zenn.dev/sdb_blog/articles/01_plus_02_does_not_equal_03)
- [decimalについて](https://engineering.mercari.com/blog/entry/20201203-basis-point/)
- [decimalパッケージ](https://github.com/shopspring/decimal)
- [Event Driven Architecture](https://aws.amazon.com/what-is/eda/)
- [Kafka Client](https://docs.redpanda.com/redpanda-labs/clients/docker-go/)
